# Fibre packer - first draft

*Author: Vedrana Andersen Dahl (vand@dtu.dk)*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vedranaa/fibre-pack/blob/main/fibre_packer_demo.ipynb)



In [1]:
import torch
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import os
if not os.path.isfile('fibre_packer.py'):
    !wget 'https://raw.githubusercontent.com/vedranaa/fibre-pack/main/fibre_packer.py' -q
import fibre_packer as fp

The function for optimizing a single slice points (below) is still work in progress. Weighting of the different losses, and stopping criteria are not yet implemented.

In [ ]:
def optimize_slice_points(p, radii, c, iters=2000):
    delta = 0.01 * radii.mean()
    n = 3
    N = len(radii)
    min_d = fp.minimal_distance(radii)
    p.requires_grad = True
    p.to(torch_device)
    optimizer = torch.optim.Adam([p], lr=0.1)
    loss_contributions = []
    progress_bar = tqdm(range(iters), bar_format='{desc:<100}{bar}{n_fmt}/{total_fmt}')
    for iter in progress_bar:  
        optimizer.zero_grad()   
        d = fp.pairwise_distance(p)
        overlap = fp.overlap_penalty(d, min_d, delta)
        protrusion = fp.protrusion_penalty(p, radii, c)
        separation = fp.separation_penalty(d, radii, n, delta)
        loss = overlap + N * protrusion + 1/N * separation
        loss.backward()
        optimizer.step()
        loss_contributions.append((overlap.item(), N * protrusion.item(), 1/N * separation.item()))
        progress_bar.set_description(f"Overlap {overlap:.2f}, " + 
            f"protrusion {protrusion:.2f}", refresh=True)
    
    p = p.detach()
    overlap = fp.overlap_penalty(d, min_d)
    protrusion = fp.protrusion_penalty(p, radii, c)
    loss_contributions = {k:list(v) for k, v in 
            zip(['overlap', 'protrusion', 'separation'], zip(*loss_contributions))}
    return p, (overlap, protrusion), loss_contributions

Choosing the size of the problem.


In [3]:
# Parameters to decide on
domain_radius = 40  # Domain radius
fibre_radius_mean = 2  # Mean fibre radius
fibre_volume_fraction = 70  # Desired fibre volume fraction
number_slices = 20 # Number of slices to generate

# Parameters calculated from the above
fibre_radius_sigma = 0.1 * fibre_radius_mean  # Standard deviation of fibre radius - TODO repair the function 
radii = fp.initialize_radii(domain_radius, fibre_volume_fraction, fibre_radius_mean, fibre_radius_sigma)
N = len(radii)  # Number of fibres

# TODO remove legend from histogram
fp.show_radii_distribution(radii)

In [4]:
p0 = fp.initialize_slice_points(domain_radius - fibre_radius_mean, N)
fp.show_slice(p0, radii, domain_radius, title='First slice (p0) initial')

In [5]:
torch_device = fp.select_device()
p0, (overlap, protrusion), losses = optimize_slice_points(p0, radii, domain_radius)
fp.show_slice(p0, radii, domain_radius, 
    title=f'First slice (p0) optimized. Overlap {overlap:.2f}, protrusion {protrusion:.2f}')

Using device mps


                                                                                                              …

In [6]:
fp.show_losses(losses)

In [7]:
pZ = p0.clone()
pZ = fp.rotate_bundle(pZ, radii, (domain_radius/2, 0), domain_radius/2.5, -torch.pi/2)
pZ = fp.rotate_bundle(pZ, radii, (-domain_radius/2, 0), domain_radius/2, -torch.pi/3)
pZ = fp.rotate_bundle(pZ, radii, (0, 0), domain_radius, torch.pi/4)
pz = fp.swap_points(pZ)

fp.show_slice(pZ, radii, domain_radius, title='Last slice (pZ) initial')

In [8]:
pZ, (overlap, protrusion), losses = optimize_slice_points(pZ, radii, domain_radius)
fp.show_slice(pZ, radii, domain_radius, 
    title=f'Last slice (pZ) optimized. Overlap {overlap:.2f}, protrusion {protrusion:.2f}')

                                                                                                              …

In [10]:
fp.show_losses(losses)

In [11]:
configuration = fp.interpolate_configuration(p0, pZ, number_slices)
fp.show_3D_configuration(configuration, title='Initial configuration')
fp.animate_configuration(configuration, title='Initial configuration')

In [12]:
N = len(radii)
min_d = fp.minimal_distance(radii)
delta = 0.01 * radii.mean()
configuration.requires_grad = True
configuration.to(torch_device)
optimizer = torch.optim.Adam([configuration], lr=0.1)
loss_contributions = []
iters = 2000
progress_bar = tqdm(range(iters), bar_format='{desc:<100}{bar}{n_fmt}/{total_fmt}')
for iter in progress_bar:  
    optimizer.zero_grad()   
    d = fp.pairwise_distance(configuration)
    overlap = fp.overlap_penalty(d, min_d, delta)
    protrusion = fp.protrusion_penalty(configuration, radii, domain_radius)
    stretching = fp.stretching_penalty(configuration)
    bending = fp.bending_penalty(configuration)
    boundary = fp.boundary_penalty(configuration, p0, pZ)
    loss = overlap + N * protrusion + 1/N * stretching + 2/N * bending + N * boundary
    loss.backward()
    optimizer.step()
    loss_contributions.append((overlap.item(), N * protrusion.item(), 
        1/N * stretching.item(), 2/N * bending.item(), N * boundary.item()))
    progress_bar.set_description(f"Over. {overlap.item():.2f}, " + 
            f"prot. {protrusion.item():.1f}, " +
            f"stre. {stretching.item():.1f}, " +
            f"bend. {bending.item():.1f}, " +
            f"boun. {boundary.item():.1f}",
            refresh=True)

loss_contributions = {k:list(v) for k, v in 
        zip(['overlap', 'protrusion', 'stretching', 'bending', 'boundary'], zip(*loss_contributions))}

fp.show_3D_configuration(configuration, title='Optimized configuration')

                                                                                                              …

In [13]:
fp.show_losses(loss_contributions)

In [15]:
fp.animate_slices(configuration, radii, domain_radius, title='Optimized configuration')